## 包引入与变量准备

In [1]:
import sys
import csv
sys.path.append("./global")
from helper import Timer, usetime, OFFLINE, cp

In [2]:
# 原始训练与测试数据位置
if OFFLINE:
    ori_train_file = "./legacy/data/train.csv"
    ori_test_file = "./legacy/data/test.csv"
else:
    ori_train_file = "./legacy/data/train.csv"
    ori_test_file = "./legacy/data/test.csv"
    # ori_train_file = "/home/kesci/input/bytedance/first-round/train.csv"
    # ori_test_file = "/home/kesci/input/bytedance/first-round/test.csv"

In [3]:
# Stage0阶段
stage0_output_dir = "./stage1/input/"
## 训练数据相关
if OFFLINE:
    train_file = stage0_output_dir + "train.csv"
else:
    random_sample = False
    if random_sample:
        random_rate = 0.05
        train_file = stage0_output_dir + "train_random_005.csv"
    else:
        random_num = 5000000
        # 默认取前500w条
        train_file = stage0_output_dir + "train_front_500w.csv"
## 测试数据相关
test_file = stage0_output_dir + "test.csv"
if OFFLINE:
    test_label_file = stage0_output_dir + "test_label.csv"

In [4]:
# Stage1阶段
state1_dir = "./stage1/"
stage1_code_dir = state1_dir + "code/"
stage1_input_dir = state1_dir + "input/"
stage1_output_dir = state1_dir + "output/"

In [5]:
# Stage2阶段
state2_dir = "./stage2/"
stage2_input_dir = state2_dir + "input/"
stage2_output_dir = state2_dir + "output/"

## Stage0阶段

这一阶段主要是数据准备，且只需要运行一次。

In [6]:
from stage0.code.data_prepare import RandomSampleCSV, SampleCSV

In [7]:
# # 训练数据准备
# if OFFLINE:
#     cp(ori_train_file, train_file)
# else:
#     if random_sample:
#         # 随机从一亿数据中采样rate比例的数据来训练
#         RandomSampleCSV(source_csv=ori_train_file, save_file=train_file, rate=random_rate)
#     else:
#         # 取前c条数据来训练   
#         SampleCSV(source_csv=ori_train_file, save_file=train_file, count=random_num)        

In [8]:
# # 测试数据准备
# if OFFLINE:
#     cp(ori_test_file, test_file)
#     cp("./legacy/data/test_label.csv", "./stage1/input/test_label.csv")
# else:
#     cp(ori_test_file, test_file)

## Stage1阶段

### 提取词向量

#### Word2vec

1. 生成Word2vec可直接训练的sentences文件
2. 训练word2vec

In [9]:
# ProcessForTrainWord2vec(train_file_data, stage1_input_dir+"word2vec_sentences.txt")

In [10]:
# !cd stage1/code; python build_word2vec.py -f "../input/word2vec_sentences.txt" -d "../output/" -m "word2vec.model"

#### FastText

FastText的使用：https://blog.csdn.net/ymaini/article/details/81489599

1. 生成适合fastText训练的文本格式
2. 有监督训练

In [11]:
# ProcessForTrainFastText(train_file_data, stage1_input_dir+"fastText_labeled_content.txt", add_label=True)

In [12]:
# !cd stage1/code; python build_fastText.py -f "../input/fastText_labeled_content.txt" -d "../output/" -s "fastText_supervised.bin"

### 特征抽取

In [13]:
from stage1.code.feature_text import ExtractTextFeature
from helper import ORI_TRAIN_NAMES, ORI_TEST_NAMES, ORI_TRAIN_DTYPE, ORI_TEST_DTYPE
feature_prefix = "v3"

#### Text Mining Featues

In [14]:
# 提取train的特征
# ExtractTextFeature(train_file, save_dir=stage1_output_dir, prefix="train_{}".format(feature_prefix), 
#                    names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, process_chunkly=False,
#                    drop_cols=['query', 'title', 'label'])

In [15]:
# 提取test的特征
# ExtractTextFeature(test_file, save_dir=stage1_output_dir, prefix="test_{}".format(feature_prefix), 
#                    names=ORI_TEST_NAMES, dtype=ORI_TEST_DTYPE, process_chunkly=False,
#                    drop_cols=['query', 'title'])

#### Embedding Features

In [16]:
# ! cd stage1/code; python feature_embed.py -f "../input/train_01_0607.csv" -p "train_v1" -d "../output" -b "true" -e "word2vec"
# ! cd stage1/code; python feature_embed.py -f "../input/test_copy.csv" -p "test_v1" -d "../output" -b "true" -e "word2vec" -t "false"

### Prepare for Stage2

本节是将一些stage1产生的结果进过处理后放到stage2的input里面去。

In [17]:
from stage1.code.prepare_for_stage2 import ExtractTrainLabel, ConvertCSVToNPY
# stage1
## text feature
train_text_feature_stage1 = stage1_output_dir + "train_{}_feature_text.csv".format(feature_prefix)
test_text_feature_stage1 = stage1_output_dir + "test_{}_feature_text.csv".format(feature_prefix)
# stage2
## text feature
train_text_feature_stage2 = stage2_input_dir + "train_{}_feature_text.npy".format(feature_prefix)
test_text_feature_stage2 = stage2_input_dir + "test_{}_feature_text.npy".format(feature_prefix)
train_labels_stage2 = stage2_input_dir+"train_labels.npy"
test_labels_stage2 = None

In [18]:
# 提取训练集的标签并单独存储
ExtractTrainLabel(train_file, train_labels_stage2)
if OFFLINE:
    test_labels_stage2 = stage2_input_dir+"test_labels.npy"
    ExtractTrainLabel(test_label_file, test_labels_stage2)

----->Started 'Extract label' block...
Part of labels: [1, 0, 1, 0, 1, 0, 0, 1, 0, 0]
Label file saved to ./stage2/input/train_labels.npy
----->Finished 'Extract label' block, time used: 0.03s.
----->Started 'Extract label' block...
Part of labels: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Label file saved to ./stage2/input/test_labels.npy
----->Finished 'Extract label' block, time used: 0.0s.


In [19]:
# 将特征转化为numpy格式便于读取
ConvertCSVToNPY(train_text_feature_stage1, train_text_feature_stage2)
ConvertCSVToNPY(test_text_feature_stage1, test_text_feature_stage2)

----->Started 'Convert CSV To NPY' block...
Part of rows: [[1.0, 1.0, 3.6635616461296463, 2.9444389791664403, 0.12244897959183673, 0.06521739130434782, 0.2992063492063493, 0.44801587301587287, 3.6375861597263857, 2.8903717578961645, 0.037037037037037035, 0.018867924528301886, 0.29472348543555965, 0.3854057102423114, 3.6109179126442243, 2.833213344056216, 0.0, 0.0, 0.290305992782974, 0.3487576082394259], [1.0, 2.0, 3.6635616461296463, 2.772588722239781, 0.25, 0.14285714285714285, 0.3268044324648097, 0.4362084456424078, 3.6375861597263857, 2.70805020110221, 0.0784313725490196, 0.04081632653061224, 0.32240058416529005, 0.3875202972088785, 3.6109179126442243, 2.6390573296152584, 0.0, 0.0, 0.3097278527447116, 0.3628817581667688]]
Npy file saved to ./stage2/input/train_v3_feature_text.npy
----->Finished 'Convert CSV To NPY' block, time used: 0.22s.
----->Started 'Convert CSV To NPY' block...
Part of rows: [[2266.0, 3.0, 1.6094379124341003, 2.0794415416798357, 0.5454545454545454, 0.375, 0.545

## Stage2阶段

In [20]:
from stage2.code.run_hyperopt import LoadDataset, run_lgb_gbdt

/Users/niudong/workon_home/py3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [21]:
# 加载数据集
LoadDataset(in_train_feature=train_text_feature_stage2, 
            in_train_label=train_labels_stage2, 
            in_test_feature=test_text_feature_stage2,
            in_test_label=test_labels_stage2)

-------------------------------------
[Runing function]: [_load_train] 
[Used Time]: 0:00:00.002905
-------------------------------------
-------------------------------------
[Runing function]: [_load_test] 
[Used Time]: 0:00:00.001622
-------------------------------------
X: [[1.         1.         3.66356165 2.94443898 0.12244898 0.06521739
  0.29920635 0.44801587 3.63758616 2.89037176 0.03703704 0.01886792
  0.29472349 0.38540571 3.61091791 2.83321334 0.         0.
  0.29030599 0.34875761]
 [1.         2.         3.66356165 2.77258872 0.25       0.14285714
  0.32680443 0.43620845 3.63758616 2.7080502  0.07843137 0.04081633
  0.32240058 0.3875203  3.61091791 2.63905733 0.         0.
  0.30972785 0.36288176]] (15000, 20)
y: [1 0] (15000,)
Test x: [[2.26600000e+03 3.00000000e+00 1.60943791e+00 2.07944154e+00
  5.45454545e-01 3.75000000e-01 5.45454545e-01 6.06060606e-01
  1.38629436e+00 1.94591015e+00 4.44444444e-01 2.85714286e-01
  4.44444444e-01 5.49019608e-01 1.09861229e+00 1.791759

In [22]:
run_lgb_gbdt(eval_num=10)

{                                                   
 "bagging_fraction": 0.8,
 "bagging_freq": 18.0,
 "feature_fraction": 0.6000000000000001,
 "learning_rate": 0.04,
 "max_bin": 15,
 "min_data_in_leaf": 100.0,
 "num_iterations": 450.0,
 "num_leaves": 75.0,
 "num_threads": 4
}
-------------------------------------               
[Runing function]: [fit]                            
[50]	valid's auc: 0.621929	valid's binary_logloss: 0.549063
[100]	valid's auc: 0.618696	valid's binary_logloss: 0.550485
[150]	valid's auc: 0.612464	valid's binary_logloss: 0.553518
[200]	valid's auc: 0.60596	valid's binary_logloss: 0.557775
[250]	valid's auc: 0.601375	valid's binary_logloss: 0.560434
[300]	valid's auc: 0.599313	valid's binary_logloss: 0.56328
[350]	valid's auc: 0.598177	valid's binary_logloss: 0.566207
[400]	valid's auc: 0.596993	valid's binary_logloss: 0.568376
[450]	valid's auc: 0.595954	valid's binary_logloss: 0.571048
[Used Time]: 0:00:02.039313                         
-----------------

-------------------------------------                                        
[Runing function]: [fit]                                                     
[50]	valid's auc: 0.609752	valid's binary_logloss: 0.552642                  
[100]	valid's auc: 0.600189	valid's binary_logloss: 0.560197                 
[150]	valid's auc: 0.582715	valid's binary_logloss: 0.569479                 
[200]	valid's auc: 0.580789	valid's binary_logloss: 0.573518                 
[250]	valid's auc: 0.562196	valid's binary_logloss: 0.583289                 
[300]	valid's auc: 0.562867	valid's binary_logloss: 0.588119                 
[Used Time]: 0:00:00.496787                                                  
-------------------------------------                                        
-------------------------------------                                        
[Runing function]: [fit]                                                     
[50]	valid's auc: 0.604425	valid's binary_logloss: 0.556206     

Test AUC:0.604470796374523                                                   
{                                                                            
 "bagging_fraction": 0.6000000000000001,
 "bagging_freq": 12.0,
 "feature_fraction": 0.8,
 "learning_rate": 0.1,
 "max_bin": 1023,
 "min_data_in_leaf": 30.0,
 "num_iterations": 400.0,
 "num_leaves": 40.0,
 "num_threads": 4
}
-------------------------------------                                        
[Runing function]: [fit]                                                     
[50]	valid's auc: 0.590006	valid's binary_logloss: 0.5669                    
[100]	valid's auc: 0.571213	valid's binary_logloss: 0.594371                 
[150]	valid's auc: 0.566485	valid's binary_logloss: 0.601097                 
[200]	valid's auc: 0.573338	valid's binary_logloss: 0.60113                  
[250]	valid's auc: 0.571531	valid's binary_logloss: 0.613118                 
[300]	valid's auc: 0.567565	valid's binary_logloss: 0.62625              

[Used Time]: 0:00:01.723323                                                  
-------------------------------------                                        
-------------------------------------                                        
[Runing function]: [fit]                                                     
[50]	valid's auc: 0.610242	valid's binary_logloss: 0.635192                  
[100]	valid's auc: 0.607713	valid's binary_logloss: 0.678493                 
[150]	valid's auc: 0.60613	valid's binary_logloss: 0.735779                  
[200]	valid's auc: 0.60017	valid's binary_logloss: 0.768677                  
[Used Time]: 0:00:01.216202                                                  
-------------------------------------                                        
Valid AUC:0.5543711995952258                                                 
-------------------------------------                                        
[Runing function]: [fit]                                        

[150]	valid's auc: 0.581366	valid's binary_logloss: 0.573063                 
[200]	valid's auc: 0.571801	valid's binary_logloss: 0.581767                 
[250]	valid's auc: 0.565803	valid's binary_logloss: 0.590166                 
[300]	valid's auc: 0.562796	valid's binary_logloss: 0.595101                 
[350]	valid's auc: 0.55951	valid's binary_logloss: 0.600879                  
[400]	valid's auc: 0.559187	valid's binary_logloss: 0.606196                 
[Used Time]: 0:00:01.696771                                                  
-------------------------------------                                        
-------------------------------------                                        
[Runing function]: [fit]                                                     
[50]	valid's auc: 0.623184	valid's binary_logloss: 0.548497                  
[100]	valid's auc: 0.616933	valid's binary_logloss: 0.552465                 
[150]	valid's auc: 0.609732	valid's binary_logloss: 0.557487    

[Runing function]: [fit]                                                     
[50]	valid's auc: 0.608872	valid's binary_logloss: 0.551657                  
[100]	valid's auc: 0.60525	valid's binary_logloss: 0.555975                  
[150]	valid's auc: 0.595959	valid's binary_logloss: 0.562583                 
[200]	valid's auc: 0.589521	valid's binary_logloss: 0.569731                 
[250]	valid's auc: 0.584457	valid's binary_logloss: 0.57442                  
[300]	valid's auc: 0.581017	valid's binary_logloss: 0.580596                 
[350]	valid's auc: 0.580161	valid's binary_logloss: 0.584501                 
[400]	valid's auc: 0.579738	valid's binary_logloss: 0.589859                 
[450]	valid's auc: 0.573398	valid's binary_logloss: 0.59641                  
[500]	valid's auc: 0.572945	valid's binary_logloss: 0.601824                 
[Used Time]: 0:00:01.781857                                                  
-------------------------------------                           